In [ ]:
#=============================================================================================================
"""
This script performs a single-phase, multi-objective optimization using the Adaptive Weighted Product method. 
The objectives are to maximize the final total mass of the vehicle and the final downrange distance.

Initially, it generates a Pareto front using uniform weights. Then, it refines the results by adaptively 
adjusting the weight distribution to better capture the trade-offs between objectives.

"""
# ============================================================================================================

from pyomo.dae import ContinuousSet, DerivativeVar, Integral
from pyomo.environ import ConcreteModel, TransformationFactory, Var, \
                          NonNegativeReals, Constraint, ConstraintList, \
                          SolverFactory, Objective, cos, sin, tan, minimize, maximize,  \
                          NonNegativeReals, NegativeReals, Param, sqrt 
from pyomo.environ import *
from pyomo.dae import *
from math import isclose
import numpy as np
import matplotlib.pyplot as plt
import csv
import logging
from MAV_Single import MAV
from pyomo.environ import Suffix, ConcreteModel, Var, NonNegativeReals, \
    Constraint, Objective, SolverFactory
from pyomo.util.infeasible import (
    find_infeasible_constraints,
    log_infeasible_constraints,
    find_infeasible_bounds,
    log_infeasible_bounds,
    find_close_to_bounds,
    log_close_to_bounds,
)

logging.basicConfig(level=logging.INFO) 
logger = logging.getLogger('pyomo.core')

def adaptive_weighted_sum_optimization(mav, max_iterations=7):

    mass_opt, range_at_mass_opt = solve_single_objective(mav, 'mass')
    mass_at_range_opt, range_opt = solve_single_objective(mav, 'range')
    
    # Utopia point: best achievable values
    J_U = [mass_opt , range_opt]
    
    # Nadir point: worst values among optimal solutions
    J_N = [mass_at_range_opt , range_at_mass_opt]
    
    current_solutions = run_initial_optimization(mav, J_U, J_N)
    needs_refinement, segments = check_convergence_and_lengths(current_solutions)

    iteration = 1
    print(f"\nIteration {iteration}")

    while True:
        if not needs_refinement or iteration >= max_iterations:
            break
            
        print(f"\nIteration {iteration + 1}")
        
        new_solutions = []
        for segment in segments:
            sub_solutions = sub_optimize_segment(mav, segment, J_U, J_N)
            if sub_solutions:
                new_solutions.extend(sub_solutions)
        
        all_solutions = current_solutions + new_solutions
        current_solutions = filter_solutions(all_solutions)

        needs_refinement, segments = check_convergence_and_lengths(current_solutions)
        
        iteration += 1
    
    return current_solutions

# Step 1 calculate the single objective optimal points as well as the normals
def solve_single_objective(mav, objective_type='mass'):

    if hasattr(mav.m, 'objective'):
        mav.m.del_component('objective')
    
    if objective_type == 'mass':
        mav.m.objective = Objective(expr=mav.m.mass, sense=maximize)
    else:
        mav.m.objective = Objective(expr=mav.m.range, sense=maximize)

    # Solve
    solver = SolverFactory('ipopt')
    solver.options["halt_on_ampl_error"] = "yes"
    solver.options['tol'] = 1e-7
    solver.options['dual_inf_tol'] = 1e-7
    solver.options['constr_viol_tol'] = 1e-7
    solver.options["max_iter"] = 1200
    solver.options["linear_scaling_on_demand"] = "yes"
    solver.options['nlp_scaling_method'] = 'gradient-based'
    solver.options['linear_solver'] = "ma27"
    solver.options["ma27_pivtol"] = 1e-7
    solver.options['acceptable_tol'] = 1e-7
    results = solver.solve(mav.m, tee=True, keepfiles=True, logfile="log_check.log")

    return value(mav.m.mass), value(mav.m.range)

# Step 2 perform multiobjective optimization using the usutal weighted-sum approach
def run_initial_optimization(mav, J_U, J_N):

    final_mass_values = []
    final_downrange_values = []
    n_initial = 5
    delta_lambda = 1.0 / n_initial
    weights = [(i * delta_lambda, 1.0 - i * delta_lambda) 
              for i in range(n_initial + 1)]
    
    solutions = []
    mav.m.del_component(mav.m.C_range)
    mav.m.del_component(mav.m.C_mass)
    mav.m.del_component(mav.m.range)
    mav.m.del_component(mav.m.mass)
    
    for w1, w2 in weights:

        if hasattr(mav.m, 'objective'):
            mav.m.del_component('objective')
        
        mav.m.range = Var(bounds=(J_N[1] + 0.0005, None))
        mav.m.mass = Var(bounds=(J_N[0] + 0.0005, None))
        
        mav.m.C_range = Constraint(expr=mav.m.range == (mav.m.x_1[1]))
        mav.m.C_mass = Constraint(expr=mav.m.mass == (mav.m.mass_1[1]))
    
        # Create normalized objectives
        normalized_mass = ((mav.m.mass) - J_N[0]) / (J_U[0] - J_N[0])
        normalized_range = ((mav.m.range) - J_N[1]) / (J_U[1] - J_N[1])
        
        mav.m.objective = Objective(expr=(normalized_mass**w1 * normalized_range**w2), sense=maximize
        )

        # Solve
        solver = SolverFactory('ipopt')
        solver.options["halt_on_ampl_error"] = "yes"
        solver.options['tol'] = 1e-8
        solver.options['dual_inf_tol'] = 1e-8
        solver.options['constr_viol_tol'] = 1e-8
        solver.options["max_iter"] = 1200
        solver.options["linear_scaling_on_demand"] = "yes"
        solver.options['nlp_scaling_method'] = 'gradient-based'
        solver.options['linear_solver'] = "ma27"
        solver.options["ma27_pivtol"] = 1e-8
        solver.options['acceptable_tol'] = 1e-8

        results = solver.solve(mav.m, tee=False)
        if (results.solver.status == SolverStatus.ok and 
        results.solver.termination_condition == TerminationCondition.optimal):
            solution =  (value(normalized_mass), value(normalized_range), (w1, w2))
        if solution:
            solutions.  append(solution)
        final_mass_values.append(value(normalized_mass))
        final_downrange_values.append(np.sqrt(value(normalized_range)**2) / 1e3) 
        mav.m.del_component(mav.m.C_range)
        mav.m.del_component(mav.m.range)
        mav.m.del_component(mav.m.mass)
        mav.m.del_component(mav.m.C_mass)
    labels = [f'$W_r={w2},W_m={w1}$' for w1, w2 in weights]

    # Plot Pareto Front
    markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'H']
    plt.figure()

    for i in range(len(final_mass_values)):
        plt.scatter(final_downrange_values[i], final_mass_values[i], marker=markers[i % len(markers)], color='black', label=f'Point {i+1}')
        if i == 0:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(-40, 10), ha='center')
        elif i == 1:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(-50, 0), ha='center')
        elif i == 2:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(50, -10), ha='center')
        elif i == 3:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(30, -13), ha='center')
        elif i == 4:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(40, 3), ha='center')

    plt.xlabel('Downrange [km]')
    plt.ylabel('Total Vehicle Mass [kg]')
    plt.legend()
    plt.grid(True)
    plt.show()

    print('solutions', solutions)
    return filter_solutions(solutions)

# Step 3 compute the length between neighboring solutions and delete overlapping solutions (euclidean distance < epsilon)
def filter_solutions(solutions, epsilon=0.025):
    filtered_solutions = []
    
    flat_solutions = []
    for sol in solutions:
        if isinstance(sol, list):
            flat_solutions.extend(sol)
        elif isinstance(sol, tuple) and len(sol) == 3:
            mass, range_val, weight = sol
            if not isinstance(weight, tuple):
                weight = (weight, 1.0 - weight)
            flat_solutions.append((mass, range_val, weight))
    for sol1 in flat_solutions:
        is_unique = True
        for sol2 in filtered_solutions:
            dist = calculate_euclidean_distance(
                (sol1[0], sol1[1]),
                (sol2[0], sol2[1])
            )
            if dist < epsilon:
                is_unique = False
                break
        if is_unique:
            filtered_solutions.append(sol1)
    print('filtered solutions', filtered_solutions)
    return filtered_solutions 

def calculate_euclidean_distance(point1, point2):
    return sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Step 4 Determine the number of further refinements in each of the regions
def check_convergence_and_lengths(solutions, max_length=0.05, C=2.0):

    solutions_list = solutions 
    if len(solutions) < 2:
        return False, []
   
    solutions_list.sort(key=lambda x: x[1])
    lengths, avg_length = calculate_segment_lengths(solutions_list)
   
    needs_refinement = False
    segments_to_refine = []
  
    for i, length in enumerate(lengths):
        if length > max_length:
            needs_refinement = True
            n_i = round(C * length / avg_length)
            
            # Only proceed if n_i > 1 as per Step 5
            if n_i > 1:
                point1 = (solutions_list[i][0], solutions_list[i][1])
                point2 = (solutions_list[i+1][0], solutions_list[i+1][1])
                delta1, delta2 = calculate_segment_offsets(point1, point2)
               
                segments_to_refine.append({
                    'index': i,
                    'start_point': solutions_list[i],
                    'end_point': solutions_list[i + 1],
                    'length': length,
                    'refinements': n_i, 
                    'offsets': (delta1, delta2)
                })
    return needs_refinement, segments_to_refine

# Step 5 Calculate the length segment length to determine if ni < 1
def calculate_segment_lengths(solutions):
    lengths = []
    for i in range(len(solutions) - 1):
        point1 = (solutions[i][0], solutions[i][1])
        point2 = (solutions[i + 1][0], solutions[i + 1][1])
        length = calculate_euclidean_distance(point1, point2)
        lengths.append(length)
    
    avg_length = sum(lengths) / len(lengths) if lengths else 0
    return lengths, avg_length

# Step 6 Determine offset distances from the two end points of each segment
def calculate_segment_offsets(point1, point2, delta_f=0.05):

    P1x, P1y = point1
    P2x, P2y = point2
    
    theta = np.arctan2(-(P2y - P1y), (P2x - P1x))

    delta1 = delta_f * np.cos(theta)
    delta2 = delta_f * np.sin(theta)
    
    return delta1, delta2

def determine_refinements(lengths, avg_length, C=2.0):
    """
    Determine number of refinements for each segment using equation (8)
    Args:
        lengths: List of segment lengths
        avg_length: Average segment length
        C: Algorithm constant
    Returns:
        List of number of refinements needed for each segment
    """
    refinements = []
    for length in lengths:
        n_i = round(C * length / avg_length)
        refinements.append(max(0, n_i))  # Ensure non-negative
    return refinements


    
    return needs_refinement, segments_to_refine

# Step 7 Impose additional inequality constraints and conduct sub-optimization
def sub_optimize_segment(mav, segment, J_U, J_N):

    start_point = segment['start_point']
    end_point = segment['end_point']
    delta1, delta2 = segment['offsets']
    n_i = segment['refinements']
    
    delta_lambda = 1.0 / n_i
    sub_solutions = []
    lambda_values = np.arange(0, 1 + delta_lambda, delta_lambda)
 
    for lambda_i in lambda_values:

        if hasattr(mav.m, 'objective'):
            mav.m.del_component('objective')

        mav.m.range = Var(bounds=(J_N[1] + 0.0005, None))
        mav.m.mass = Var(bounds=(J_N[0] + 0.0005, None))
        
        mav.m.C_range = Constraint(expr=mav.m.range == (mav.m.x_1[1]))
        mav.m.C_mass = Constraint(expr=mav.m.mass == (mav.m.mass_1[1]))
    
        normalized_mass = ((mav.m.mass) - J_N[0]) / (J_U[0] - J_N[0])
        normalized_range = ((mav.m.range) - J_N[1]) / (J_U[1] - J_N[1])

        mav.m.ineq_mass = Constraint(
            expr=normalized_mass <= start_point[0] - delta1
        )
        mav.m.ineq_range = Constraint(
            expr=normalized_range >= start_point[1] - delta2
        )
        
        mav.m.objective = Objective(
            expr=((normalized_mass**lambda_i)  * (normalized_range ** (1 - lambda_i)) ),
            sense=maximize
        )
        
        # Solve
        solver = SolverFactory('ipopt')
        solver.options["halt_on_ampl_error"] = "yes"
        solver.options['tol'] = 1e-7
        solver.options['dual_inf_tol'] = 1e-7
        solver.options['constr_viol_tol'] = 1e-7
        solver.options["max_iter"] = 1200
        solver.options["linear_scaling_on_demand"] = "yes"
        solver.options['nlp_scaling_method'] = 'gradient-based'
        solver.options['linear_solver'] = "ma27"
        solver.options["ma27_pivtol"] = 1e-7
        solver.options['acceptable_tol'] = 1e-7
        results = solver.solve(mav.m, tee=False)
        
        if (results.solver.status == SolverStatus.ok and 
        results.solver.termination_condition == TerminationCondition.optimal):
            sub_solutions.append((
                value(normalized_mass),
                value(normalized_range),
                lambda_i
            ))
        mav.m.del_component(mav.m.ineq_mass)
        mav.m.del_component(mav.m.ineq_range)
        mav.m.del_component(mav.m.C_range)
        mav.m.del_component(mav.m.range)
        mav.m.del_component(mav.m.mass)
        mav.m.del_component(mav.m.C_mass)
    
    return sub_solutions

def main():

    # Initial and final conditions (optional)
    x0 = None
    y0 = None
    z0 = None

    u0 = None
    v0 = None
    w0 = None

    phi0 = None
    the0 = None
    psi0 = None
    
    p0 = None
    q0 = None
    r0 = None

    xf = None
    yf = None
    zf = None
    
    uf = None
    vf = None
    wf = None

    phif = None
    thef = None 
    psif = None
    
    pf = None
    qf = None
    rf = None

    # Initialize lists to store results
    final_mass_values = []
    final_downrange_values = []

    # Weightings 
    W_Obj1 = [0.0, 0.0, 0.2, 0.2, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8, 1.0, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
    W_Obj2 = [2.0, 1.0, 1.8, 0.8, 0.6, 1.6, 0.4, 1.4, 1.2, 0.2, 0.0, 1.0, 0.8, 0.6, 0.4, 0.2, 0.0]

    for weight in range(1):
        for warm in range(1):

            # Create mav vehicle
            mav = MAV(x0=x0, y0=y0, z0=z0, u0=u0, v0=v0, w0=w0, phi0=phi0, the0=the0, psi0=psi0, p0=p0, q0=q0, r0=r0, \
                    xf=xf, yf=yf, zf=zf, uf=uf, vf=vf, wf=wf, phif=phif, thef=thef, psif=psif, pf=pf, qf=qf, rf=rf)
        
            # Boundary Conditions
            mav.m.BCs_con = ConstraintList(rule=mav.BCs)
            
            # Phase 1
            n1 = [1] # flag for Phase 1

            mav.m.Q_dmpdot_dtau_Con1         = Constraint(n1, mav.m.t1, rule=mav.Q_dmass_dtau)

            mav.m.Q_dx_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dx_dtau)
            mav.m.Q_dy_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dy_dtau)
            mav.m.Q_dz_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dz_dtau)

            mav.m.Q_du_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_du_dtau)
            mav.m.Q_dv_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dv_dtau)
            mav.m.Q_dw_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dw_dtau)
            
            mav.m.Q_dp_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dp_dtau)
            mav.m.Q_dq_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dq_dtau)
            mav.m.Q_dr_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dr_dtau)

            mav.m.Q_dq0_dtau_Con1            = Constraint(n1, mav.m.t1, rule=mav.Q_dq0_dtau)
            mav.m.Q_dq1_dtau_Con1            = Constraint(n1, mav.m.t1, rule=mav.Q_dq1_dtau)
            mav.m.Q_dq2_dtau_Con1            = Constraint(n1, mav.m.t1, rule=mav.Q_dq2_dtau)
            mav.m.Q_dq3_dtau_Con1            = Constraint(n1, mav.m.t1, rule=mav.Q_dq3_dtau)

            mav.m.Q_massdot_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_massdot)

            mav.m.Q_pdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_pdot)
            mav.m.Q_qdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_qdot)   
            mav.m.Q_rdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_rdot)

            mav.m.Q_udot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_udot)
            mav.m.Q_vdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_vdot)   
            mav.m.Q_wdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_wdot)

            mav.m.Q_q0dot_Con1               = Constraint(n1, mav.m.t1, rule=mav.Q_q0dot)
            mav.m.Q_q1dot_Con1               = Constraint(n1, mav.m.t1, rule=mav.Q_q1dot)
            mav.m.Q_q2dot_Con1               = Constraint(n1, mav.m.t1, rule=mav.Q_q2dot)
            mav.m.Q_q3dot_Con1               = Constraint(n1, mav.m.t1, rule=mav.Q_q3dot)

           
            mav.m.Q_u_Con1                   = Constraint(n1, mav.m.t1, rule=mav.Q_u)
            mav.m.Q_v_Con1                   = Constraint(n1, mav.m.t1, rule=mav.Q_v)
            mav.m.Q_w_Con1                   = Constraint(n1, mav.m.t1, rule=mav.Q_w)

            mav.m.Q_normality_Con1           = Constraint(n1, mav.m.t1, rule=mav.Q_normality)
            

            mav.m.range = Var(bounds=(0.01, None))
            mav.m.mass = Var(bounds=(0.01, None))
            
            mav.m.C_range = Constraint(expr=mav.m.range == (mav.m.x_1[1]))
            mav.m.C_mass = Constraint(expr=mav.m.mass == (mav.m.mass_1[1]))
            if weight == 0:
                final_solutions = adaptive_weighted_sum_optimization(mav)
                if not final_solutions:
                    print("No solutions found!")
                    return None
                print("Final solutions before plotting:", final_solutions)

                masses = [sol[0] for sol in final_solutions]
                ranges = [sol[1] for sol in final_solutions]
                
                if masses and ranges:
                    plt.figure(figsize=(10, 6))
                    plt.scatter(ranges, masses, c='blue', marker='o', label='Pareto Solutions')
                    sorted_indices = np.argsort(ranges)
                    sorted_ranges = [ranges[i] for i in sorted_indices]
                    sorted_masses = [masses[i] for i in sorted_indices]
                    plt.plot(sorted_masses, sorted_ranges, 'b--', alpha=0.5)
                    plt.xlabel('Range')
                    plt.ylabel('Mass')
                    plt.title('Pareto Front - Adaptive Weighted Sum')
                    plt.grid(True)
                    plt.legend()
                    plt.show()
                    
                    print("\nFinal Pareto Solutions:")
                    for i, (mass, range_val, weights) in enumerate(final_solutions):
                        print(f"Point {i+1}:")
                        print(f"  Mass: {mass:.6f}")
                        print(f"  Range: {range_val:.6f}")
                        print(f"  Weights: {weights}")
                else:
                    print("No valid solutions to plot")

            if warm == 0:
                    final_mass_values.append(value(mav.m.mass_1[1] * mav.m.mass_scale))
                    final_downrange_values.append(np.sqrt(value(mav.m.x_1[1] * mav.m.x_scale)**2 + value(mav.m.y_1[1] * mav.m.y_scale)**2) / 1e3)    
            results_list = []

            for t1 in mav.m.t1:
                results_list.append({
                    't': t1 * value(mav.m.tf1),
                    'x': value(mav.m.x_1[t1] * mav.m.x_scale),
                    'y': value(mav.m.y_1[t1] * mav.m.y_scale),
                    'downrange': sqrt(value(mav.m.x_1[t1] * mav.m.x_scale)**2 + value(mav.m.y_1[t1] * mav.m.y_scale)**2),
                    'altitude': value(mav.m.z_1[t1] * mav.m.z_scale),
                })

            def export_to_csv(results_list, filename):
                keys = results_list[0].keys()
                with open(filename, 'w', newline='') as output_file:
                    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
                    dict_writer.writeheader()
                    dict_writer.writerows(results_list)
            export_to_csv(results_list, f'results1_{weight}.csv')

        plt.show()  

    
    labels = [f'$W_r={w2},W_m={w1}$' for w1, w2 in zip(W_Obj1, W_Obj2)]

    markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'H']
    plt.figure()

    for i in range(len(final_mass_values)):
        plt.scatter(final_downrange_values[i], final_mass_values[i], marker=markers[i % len(markers)], color='black', label=f'Point {i+1}')
        if i == 0:
            plt.annotate(labels[i % len(labels)], (final_downrange_values[i], final_mass_values[i]), textcoords="offset points", xytext=(-40, 10), ha='center')
        elif i == 1:
            plt.annotate(labels[i % len(labels)], (final_downrange_values[i], final_mass_values[i]), textcoords="offset points", xytext=(-50, 0), ha='center')
        elif i == 2:
            plt.annotate(labels[i % len(labels)], (final_downrange_values[i], final_mass_values[i]), textcoords="offset points", xytext=(50, -10), ha='center')
        elif i == 3:
            plt.annotate(labels[i % len(labels)], (final_downrange_values[i], final_mass_values[i]), textcoords="offset points", xytext=(30, -13), ha='center')
        elif i == 4:
            plt.annotate(labels[i % len(labels)], (final_downrange_values[i], final_mass_values[i]), textcoords="offset points", xytext=(40, 3), ha='center')

    plt.xlabel('Downrange [km]')
    plt.ylabel('Total Vehicle Mass [kg]')
    plt.legend()
    plt.grid(True)
    plt.show()

    return mav.m


if __name__ == '__main__':  
    m = main()
